# Sort it.

In maslite there's an alarm clock, that tracks new messages for their alarms. This "admin" should of course have as little overhead as possible, so an effective implementation is of course important.

The question is then: What IS faster?

Let's start with the baseline:

In [1]:
import heapq
from collections import defaultdict, deque
from itertools import count
import bisect
import time
import random
random.seed(43)

In [2]:
class ListAlarm(object):
    def __init__(self) -> None:
        self.alarms = []
    
    def set_alarm(self, t, msg):
        self.alarms.append((t, msg))
    
    def get_alarms(self, now):
        release = [(t,m) for t,m in self.alarms if t <= now]
        release.sort(key=lambda x:x[0])
        release = [m for t,m in release]
        self.alarms = [(t,m) for t,m in self.alarms if t > now]
        return release

In [3]:
class DefaultDictAlarm(object):
    def __init__(self) -> None:
        self.alarms = defaultdict(list)
    
    def set_alarm(self, t, msg):
        self.alarms[t].append(msg)
    
    def get_alarms(self, now):
        keys = [k for k in self.alarms.keys() if k <= now]
        release = []
        for k in sorted(keys):
            release.extend(self.alarms.pop(k))
        return release

In [4]:
class HeapqAlarm(object):
    def __init__(self) -> None:
        self.arrival_order = count()
        self.alarms = []
    
    def set_alarm(self, t, msg):
        heapq.heappush(self.alarms, (t, next(self.arrival_order), msg))
    
    def get_alarms(self, now):
        release = []
        t = -1
        while t <= now and self.alarms:
            t, o, m = heapq.heappop(self.alarms)
            if t < now:
                release.append(m)
            else:
                self.alarms.push((t,o,m))
                break
        return release

In [5]:
class InsortListAlarm(object):
    def __init__(self) -> None:
        self.arrival_order = count()
        self.alarms = []
    
    def set_alarm(self, t, msg):
        bisect.insort(self.alarms, (t, next(self.arrival_order), msg))  # https://docs.python.org/3/library/bisect.html
    
    def get_alarms(self, now):
        index = bisect.bisect(self.alarms, (now, 0, "") )
        release = self.alarms[:index]
        self.alarms = self.alarms[index:]
        return [m for t,o,m in release]

In [6]:
class InsortDequeAlarm(object):
    def __init__(self) -> None:
        self.arrival_order = count()
        self.alarms = deque()
    
    def set_alarm(self, t, msg):
        bisect.insort(self.alarms, (t, next(self.arrival_order), msg))
    
    def get_alarms(self, now):
        index = bisect.bisect(self.alarms, (now, 0, ""))
        return [m for t,o,m in [self.alarms.popleft() for _ in range(index)]]


In [7]:
# data used for comparison
alarms = 100_000
timesteps = 1000

sample = []
for i in range(alarms):
    t = round(random .random() * timesteps, 3) 
    sample.append((t, f"message no {i} for t:{t}" ))

checklist = [m for t,m in (sorted(sample, key=lambda x: x[0]))]

for cls_ in [
    ListAlarm, 
    DefaultDictAlarm, 
    HeapqAlarm,
    InsortListAlarm, 
    InsortDequeAlarm
    ]:
     
    alarm = cls_()
    start = time.perf_counter()  

    for item in sample:
        alarm.set_alarm(*item)  

    all_alarms = []
    for timestep in range(timesteps, alarms+timesteps, timesteps):  # 1000, 2000, ... 100_000, 101_000
        items = alarm.get_alarms(timestep)
        all_alarms.extend(items)

    end = time.perf_counter()
    assert all_alarms == checklist, f"{cls_.__name__} did not sort the messages"
    print(cls_.__name__, end-start)

ListAlarm 0.07317479999619536
DefaultDictAlarm 0.14283020000584656
HeapqAlarm 0.1602705999976024
InsortListAlarm 0.8610735999973258
InsortDequeAlarm 2.182989799999632


So far it looks like ListAlarm is the fastest.